# REQUIRES LARGE/GPU INSTANCE TYPE
## google/flan-t5-base toxicity score:
Mean: 0.11131308024933918 - Std: 0.26920890184712915

In [11]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=33229979648, available=31234375680, percent=6.0, used=1579479040, free=20656459776, active=811823104, inactive=9963945984, buffers=2768896, cached=10991271936, shared=847872, slab=1312268288)


# Quantitative Results with ROGUE Metric

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does give an indication to the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [12]:
%pip install --disable-pip-version-check -q \
    transformers==4.27.2 \
    datasets==2.9.0 \
    accelerate==0.17.0 \
    promptsource==0.2.3 \
    evaluate==0.4.0 \
    trl==0.4.1 \
    rouge_score==0.1.2 \
    loralib==0.1.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
sagemaker 2.145.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 6.3.0 which is incompatible.
sagemaker 2.145.0 requires PyYAML==5.4.1, but you have pyyaml 6.0 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import argparse
import csv

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

toxicity_evaluator = evaluate.load("ybelkada/toxicity", "DaNLP/da-electra-hatespeech-detection", module_type="measurement")
ds = load_dataset("OxAISH-AL-LLM/wiki_toxic", split="test")

# parser = argparse.ArgumentParser(description="Evaluate de-toxified models")
# parser.add_argument("--model_type", default="all", type=str, help="Relative path to the source model folder")
# parser.add_argument("--output_file", default="toxicity.csv", type=str, help="Relative path to the source model folder")
# parser.add_argument("--batch_size", default=64, type=int, help="Batch size")
# parser.add_argument("--num_samples", default=400, type=int, help="Number of samples")
# parser.add_argument("--context_length", default=2000, type=int, help="Number of samples")
# parser.add_argument("--max_new_tokens", default=30, type=int, help="Max new tokens for generation")
# args = parser.parse_args()

class Args:
    model_type: str
    output_file: str
    batch_size: float
    num_samples: float
    content_length: float
    max_new_tokens: str

args = Args()    
args.model_type='all'
args.output_file='toxicity.csv'
args.batch_size=64
args.num_samples=400
args.context_length=2000
args.max_new_tokens=30


if args.model_type == "all":
    MODELS_TO_TEST = [
        "t5-base",
        "google/flan-t5-base",
    ]
    
NUM_SAMPLES = args.num_samples
BATCH_SIZE = args.batch_size
output_file = args.output_file
max_new_tokens = args.max_new_tokens
context_length = args.context_length
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu"
# consider only toxic prompts
ds = ds.filter(lambda x: x["label"] == 1)

toxicities = {}

# open a csv file
file = open(f"{output_file}", "w", newline="")
writer = csv.writer(file)
# add first rows
writer.writerow(["model_id", "mean_toxicity", "std_toxicity"])


for model_id in tqdm(MODELS_TO_TEST):
    # load base LLM model and tokenizer
    if 'peft' in model_id:
        from peft import PeftModel, PeftConfig
        from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

        pretrained_model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_checkpoint, device_map={"": device}, torch_dtype=torch.bfloat16)  #, device_map="auto")

        # Load the LoRA/PEFT model
        model = PeftModel.from_pretrained(pretrained_model, model_id, device_map={"": device}, torch_dtype=torch.bfloat16)
    else:
        model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map={"": device}, torch_dtype=torch.bfloat16)
        tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    input_texts = []

    for i, example in enumerate(ds):
        # set seed
        torch.manual_seed(42)

        input_text = example["comment_text"]
        input_texts.append(input_text[:2000])

        if i > NUM_SAMPLES:
            break

        if (i + 1) % BATCH_SIZE == 0:
            inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
            inputs.input_ids = inputs.input_ids[:context_length]
            inputs.attention_mask = inputs.attention_mask[:context_length]
            outputs = model.generate(**inputs, do_sample=True, max_new_tokens=max_new_tokens, use_cache=True)
            generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            generated_texts = [
                generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)
            ]
            toxicity_score = toxicity_evaluator.compute(predictions=generated_texts)
            input_texts = []

            if model_id not in toxicities:
                toxicities[model_id] = []
            toxicities[model_id].extend(toxicity_score["toxicity"])

    # last batch
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(**inputs, do_sample=True, max_new_tokens=30)
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_texts = [generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)]
    toxicity_score = toxicity_evaluator.compute(predictions=generated_texts)
    toxicities[model_id].extend(toxicity_score["toxicity"])

    # compute mean & std using np
    mean = np.mean(toxicities[model_id])
    std = np.std(toxicities[model_id])

    # save to file
    writer.writerow([model_id, mean, std])

    # print
    print(f"Model: {model_id} - Mean: {mean} - Std: {std}")

    model = None
    torch.cuda.empty_cache()


# close file
file.close()

Found cached dataset wiki_toxic (/root/.cache/huggingface/datasets/OxAISH-AL-LLM___wiki_toxic/default/1.0.0/09a67129f85f67f22107b0190f7c32050ef0dce44afeedc6e3e0ab7ab3bd709c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/OxAISH-AL-LLM___wiki_toxic/default/1.0.0/09a67129f85f67f22107b0190f7c32050ef0dce44afeedc6e3e0ab7ab3bd709c/cache-411a66332d7a5b58.arrow
  0%|          | 0/2 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` o

Model: t5-base - Mean: 0.21037880599033087 - Std: 0.34114468286342775


Token indices sequence length is longer than the specified maximum sequence length for this model (780 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 2/2 [01:01<00:00, 30.60s/it]

Model: google/flan-t5-base - Mean: 0.10735442655624947 - Std: 0.2648808616377917


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>